In [235]:
import os
import sys
import json
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Conv2D,
    MaxPooling2D,
    Flatten,
    GlobalMaxPooling2D,
    Input,
)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import (
    categorical_crossentropy,
    binary_crossentropy,
)
from tensorflow.keras.applications import VGG19, VGG16, ResNet50, InceptionV3

import tensorflow as tf

In [23]:
train_image = os.listdir(
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/train_augm/images"
)
test_image = os.listdir(
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/test_augm/images"
)
valid_image = os.listdir(
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/valid_augm/images"
)
train_image = sorted(train_image)
test_image = sorted(test_image)
valid_image = sorted(valid_image)

In [24]:
train_label = os.listdir(
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/train_augm/labels"
)
test_label = os.listdir(
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/test_augm/labels"
)
valid_label = os.listdir(
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/valid_augm/labels"
)

train_label = sorted(train_label)
test_label = sorted(test_label)
valid_label = sorted(valid_label)

## convert them to dataframe 

In [25]:
def load_json_data(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    face = data["shapes"][0]["label"]
    if face == "face":
        face = 1
        box = data["shapes"][0]["points"]

        result = [1, box[0][0], box[0][1], box[1][0], box[1][1]]
    else:
        face = 0

        result = [0, 0, 0, 0, 0]

    return result


face = []
x1 = []
y1 = []
x2 = []
y2 = []
for i in range(len(train_label)):
    train_label[i] = load_json_data(
        "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/train_augm/labels/"
        + train_label[i]
    )
    face.append(train_label[i][0])
    x1.append(train_label[i][1])
    y1.append(train_label[i][2])
    x2.append(train_label[i][3])
    y2.append(train_label[i][4])

train_dataFrame = pd.DataFrame(
    {
        "image": train_image,
        "face": face,
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2,
    }
)

face = []
x1 = []
y1 = []
x2 = []
y2 = []

for i in range(len(test_label)):
    test_label[i] = load_json_data(
        "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/test_augm/labels/"
        + test_label[i]
    )
    face.append(test_label[i][0])
    x1.append(test_label[i][1])
    y1.append(test_label[i][2])
    x2.append(test_label[i][3])
    y2.append(test_label[i][4])

test_dataFrame = pd.DataFrame(
    {
        "image": test_image,
        "face": face,
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2,
    }
)


face = []
x1 = []
y1 = []
x2 = []
y2 = []

for i in range(len(valid_label)):
    valid_label[i] = load_json_data(
        "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/valid_augm/labels/"
        + valid_label[i]
    )
    face.append(valid_label[i][0])
    x1.append(valid_label[i][1])
    y1.append(valid_label[i][2])
    x2.append(valid_label[i][3])
    y2.append(valid_label[i][4])

valid_dataFrame = pd.DataFrame(
    {
        "image": test_image,
        "face": face,
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2,
    }
)

In [28]:
face.clear()
x1.clear()
y1.clear()
x2.clear()
y2.clear()
train_image.clear()
test_image.clear()
valid_image.clear()
train_label.clear()
test_label.clear()
valid_label.clear()

In [29]:
train_dataFrame.head()

,image,face,x1,y1,x2,y2
0,00a6fab8-7396-11ef-a98c-9c443d1ab0ba0.jpg,0,0.0,0.000000,0.000000,0.000000
1,00a6fab8-7396-11ef-a98c-9c443d1ab0ba1.jpg,1,434.0,157.000000,268.000000,0.000000
2,00a6fab8-7396-11ef-a98c-9c443d1ab0ba10.jpg,1,355.0,249.999985,189.000015,76.000008
3,00a6fab8-7396-11ef-a98c-9c443d1ab0ba11.jpg,1,217.0,256.000000,51.000015,118.000008
4,00a6fab8-7396-11ef-a98c-9c443d1ab0ba12.jpg,0,0.0,0.000000,0.000000,0.000000


In [30]:
test_dataFrame.head()

,image,face,x1,y1,x2,y2
0,e4af90cc-7395-11ef-a98c-9c443d1ab0ba0.jpg,0,0.000000,0.0,0.000000,0.000000
1,e4af90cc-7395-11ef-a98c-9c443d1ab0ba1.jpg,1,236.999985,256.0,44.000031,49.999989
2,e4af90cc-7395-11ef-a98c-9c443d1ab0ba10.jpg,1,512.000000,256.0,452.000000,49.999989
3,e4af90cc-7395-11ef-a98c-9c443d1ab0ba11.jpg,0,0.000000,0.0,0.000000,0.000000
4,e4af90cc-7395-11ef-a98c-9c443d1ab0ba12.jpg,0,0.000000,0.0,0.000000,0.000000


In [31]:
valid_dataFrame.head()

,image,face,x1,y1,x2,y2
0,e4af90cc-7395-11ef-a98c-9c443d1ab0ba0.jpg,1,512.0,79.000008,283.999969,0.0
1,e4af90cc-7395-11ef-a98c-9c443d1ab0ba1.jpg,1,512.0,256.000000,306.000000,97.0
2,e4af90cc-7395-11ef-a98c-9c443d1ab0ba10.jpg,1,442.0,256.000000,208.999969,0.0
3,e4af90cc-7395-11ef-a98c-9c443d1ab0ba11.jpg,0,0.0,0.000000,0.000000,0.0
4,e4af90cc-7395-11ef-a98c-9c443d1ab0ba12.jpg,1,512.0,238.000000,375.000000,0.0


## [Prepare data format]()

In [190]:
train_images = (
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/train_augm/images/"
    + train_dataFrame.iloc[:, 0]
)
test_images = (
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/test_augm/images/"
    + test_dataFrame.iloc[:, 0]
)
valid_images = (
    "/home/hany_jr/Ai/FaceDetectionDeepLearning/Dataset/valid_augm/images/"
    + valid_dataFrame.iloc[:, 0]
)

train_labels = train_dataFrame.iloc[:, 1:]
test_labels = test_dataFrame.iloc[:, 1:]
valid_labels = valid_dataFrame.iloc[:, 1:]

In [218]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image.set_shape((256, 512, 3))

    # normalize image

    image = tf.cast(image, tf.float32) / 255.0

    label[0].set_shape([])
    label[1].set_shape([])
    label[2].set_shape([])
    label[3].set_shape([])
    label[4].set_shape([])

    face = label[0]
    box = [label[1], label[2], label[3], label[4]]
    return image, [face], box

In [228]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels)).map(read_image).batch(1)
)


test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.shuffle(buffer_size=len(test_labels)).map(read_image)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images, valid_labels))
valid_dataset = valid_dataset.shuffle(buffer_size=len(valid_labels)).map(read_image)

In [229]:
train_dataset.as_numpy_iterator().next()[0].shape

(1, 256, 512, 3)

In [236]:
def build_model():
    input_layer = Input(shape=(256, 512, 3))

    vgg = ResNet50(include_top=False)(input_layer)

    # Classification Model
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(10, activation="relu")(f1)
    class2 = Dense(1, activation="sigmoid")(class1)

    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(10, activation="linear")(f2)
    regress2 = Dense(4, activation="linear")(regress1)

    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker


model = build_model()
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 163s 2us/step


Model: "functional_28"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_63      │ (None, 256, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 8, 16,     │ 23,587,712 │ input_layer_63[0… │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_112 (Dense)   │ (None, 10)        │     20,490 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_114 (Dense)   │ (None, 10)        │     20,490 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_113 (Dense)   │ (None, 1)         │         11 │ dense_112[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_115 (Dense)   │ (None, 4)         │         44 │ dense_114[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,628,747 (90.14 MB)

 Trainable params: 23,575,627 (89.93 MB)

 Non-trainable params: 53,120 (207.50 KB)

## [train]()

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=["binary_crossentropy", "mse"],
    metrics=["accuracy", "accuracy"],
)
model.fit(train_dataset, epochs=10)